In [ ]:
import py7zr
from zipfile import ZipFile
from random import sample
import PIL.Image as Image
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from os import listdir
from os import path
import h5py
import wget

In [ ]:
# data_path = IITD_Dataset.7z
filename = 'IITD_Dataset.7z'
# if (path.exists(filename)):
#     !rm $filename
#     print("File Removed!")
# print("Downloading Dataset...")
# wget.download(data_path, filename)
# print("Download Complete!")

with py7zr.SevenZipFile(filename, mode='r') as z:
    z.extractall()
    print("Extracted Dataset!")


In [ ]:
src_dir = 'ear/processed/221'
images_name = listdir(src_dir)
images_name_temp = []
subject_id = []
for img_ind in range(0, len(images_name)):
    if(not(images_name[img_ind]=='Thumbs.db')):
        subject_id.append(int(images_name[img_ind].split('_')[0]))
        images_name_temp.append(images_name[img_ind])
        

images_name = images_name_temp
images_name_ordered = []
subject_id_ordered = []

sub_ind = sorted(range(len(subject_id)), key=subject_id.__getitem__)   # Sorting the subject id
for pos, item in enumerate(sub_ind):
    images_name_ordered.append(images_name[item])
    subject_id_ordered.append(subject_id[item])
print(sub_ind)
images_name = images_name_ordered
subject_id = subject_id_ordered

# print(subject_id)
# print(images_name)

In [ ]:
img_ind = 0
ear_images = []
sub_labels = []
target_size = (180, 50)

for sub_ind in range (0,len(subject_id)):
    img_path = src_dir + '/' + images_name[sub_ind]
    ear_img = (plt.imread(img_path))/255

    ear_img = Image.open(img_path)
    ear_img = ear_img.resize(target_size, Image.ANTIALIAS)
    ear_img = np.asarray(ear_img).astype(np.float32)/255

    ear_images.append(ear_img)
    sub_labels.append(subject_id[sub_ind]-1)

ear_images = np.array(ear_images)
sub_labels = np.array(sub_labels)
print('Sub_Labels = ', sub_labels)
print("Shape of the dataset:", ear_images.shape)
print("Shape of the labels:", sub_labels.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(ear_images, sub_labels, test_size=0.382093316519, random_state=42)

print("Shape of the training dataset:", X_train.shape)  
print("Shape of the training labels:", y_train.shape)   
print("Shape of the testing dataset:", X_test.shape)    
print("Shape of the testing labels:", y_test.shape) 


In [ ]:
import torch
import torch.nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn.functional
import torch.optim as optim
from torchvision import models
from torchvision import transforms
# import torchsummary 
from torchinfo import summary

In [ ]:
class Pytorch_BUS_Final_Model_C1(torch.nn.Module):
    def __init__(self, num_classes=221, num_filters=8, input_shape=(180,50,1)):
        super(Pytorch_BUS_Final_Model_C1, self).__init__()
        kernel_size = 3
        #Encoder Layer 1
        self.encoder_layer1_name = 'encoder_layer1'
        self.encoder_layer1_conv = torch.nn.Conv2d(in_channels=1, out_channels=num_filters, kernel_size=kernel_size, padding='same')
        self.encoder_layer1_activation = torch.nn.ReLU()
        self.encoder_layer1_pool = torch.nn.MaxPool2d(kernel_size=(2, 2))

        #Encoder Layer 2
        self.encoder_layer2_name = 'encoder_layer2'
        self.encoder_layer2_conv = torch.nn.Conv2d(in_channels=num_filters, out_channels=num_filters*2, kernel_size=kernel_size, padding='same')
        self.encoder_layer2_activation = torch.nn.ReLU()
        self.encoder_layer2_pool = torch.nn.MaxPool2d(kernel_size=(2, 2))
        self.encoder_layer2_batchnorm = torch.nn.BatchNorm2d(num_features=num_filters*2,eps=1e-3,momentum=0.99) 

        #Encoder Layer 3
        self.encoder_layer3_name = 'encoder_layer3'   
        self.encoder_layer3_conv = torch.nn.Conv2d(in_channels=num_filters*2, out_channels=num_filters*4, kernel_size=kernel_size, padding='same')  
        self.encoder_layer3_activation = torch.nn.ReLU()
        self.encoder_layer3_pool = torch.nn.MaxPool2d(kernel_size=(2, 2))
        #self.encoder_layer3_batchnorm = torch.nn.BatchNorm2d(num_features=num_filters*4,eps=1e-3,momentum=0.99)

        #Encoder Layer 4
        self.encoder_layer4_name = 'encoder_layer4'
        self.encoder_layer4_conv = torch.nn.Conv2d(in_channels=num_filters*4, out_channels=num_filters*8, kernel_size=kernel_size, padding='same')
        self.encoder_layer4_activation = torch.nn.ReLU()
        self.encoder_layer4_pool = torch.nn.MaxPool2d(kernel_size=(2, 2))
        self.encoder_layer4_batchnorm = torch.nn.BatchNorm2d(num_features=num_filters*8,eps=1e-3,momentum=0.99)

        #Encoder Layer 5
        self.encoder_layer5_name = 'encoder_layer5'
        self.encoder_layer5_conv = torch.nn.Conv2d(in_channels=num_filters*8, out_channels=num_filters*16, kernel_size=kernel_size, padding='same')
        self.encoder_layer5_activation = torch.nn.ReLU()
        self.encoder_layer5_pool = torch.nn.MaxPool2d(kernel_size=(2, 2))
        # self.encoder_layer5_batchnorm = torch.nn.BatchNorm2d(num_features=num_filters*16,eps=1e-3,momentum=0.99)

        #Encoder Layer 6
        self.encoder_layer6_name = 'encoder_layer6'
        self.encoder_layer6_conv = torch.nn.Conv2d(in_channels=num_filters*16, out_channels=num_filters*32, kernel_size=kernel_size, padding='same')
        self.encoder_layer6_activation = torch.nn.ReLU()
        #self.encoder_layer6_pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        self.encoder_layer6_batchnorm = torch.nn.BatchNorm2d(num_features=num_filters*32,eps=1e-3,momentum=0.99)
        
        #Dense Layers
        self.fc1_flatten = torch.nn.Flatten()
        self.fc1_linear = torch.nn.Linear(32*num_filters*(input_shape[0]//(2**5))*(input_shape[1]//(2**5)), out_features= num_classes)
        self.fc1_activation = torch.nn.Softmax()

    def forward(self,x):
        #Encoder Layer 1
        out = self.encoder_layer1_conv(x)
        out = self.encoder_layer1_activation(out)   
        out = self.encoder_layer1_pool(out)

        #Encoder Layer 2
        out = self.encoder_layer2_conv(out)
        out = self.encoder_layer2_activation(out)
        out = self.encoder_layer2_pool(out)
        out = self.encoder_layer2_batchnorm(out)

        #Encoder Layer 3
        out = self.encoder_layer3_conv(out)
        out = self.encoder_layer3_activation(out)
        out = self.encoder_layer3_pool(out)
        #out = self.encoder_layer3_batchnorm(out)

        #Encoder Layer 4
        out = self.encoder_layer4_conv(out)
        out = self.encoder_layer4_activation(out)
        out = self.encoder_layer4_pool(out)
        out = self.encoder_layer4_batchnorm(out)

        #Encoder Layer 5
        out = self.encoder_layer5_conv(out)
        out = self.encoder_layer5_activation(out)
        out = self.encoder_layer5_pool(out)
        # out = self.encoder_layer5_batchnorm(out)

        #Encoder Layer 6
        out = self.encoder_layer6_conv(out)
        out = self.encoder_layer6_activation(out)
        #out = self.encoder_layer6_pool(out)
        out = self.encoder_layer6_batchnorm(out)

        #Dense Layers
        out = self.fc1_flatten(out)
        out = self.fc1_linear(out)
        out = self.fc1_activation(out)

        return out

In [ ]:
pytorch_model = Pytorch_BUS_Final_Model_C1()
print(pytorch_model)
summary(pytorch_model, input_size=(1,1, 180, 50))

In [ ]:
input_x = torch.tensor(X_train[0].reshape(1,1,180,50),device='cuda').float()
print (input_x.shape)
output = pytorch_model(input_x)
print(output.shape)
print(output)

In [136]:
# training_loader = torch.utils.data.DataLoader(torch.tensor(X_train),  batch_size=32, shuffle=True)
# validation_loader = torch.utils.data.DataLoader(X_test,  batch_size=32, shuffle=True)
training_loader = DataLoader(TensorDataset(torch.tensor(X_train), torch.tensor(y_train)), batch_size=32, pin_memory='True', pin_memory_device='cuda', shuffle=True)
validation_loader = DataLoader(TensorDataset(torch.tensor(X_test), torch.tensor(y_test)), batch_size=32, pin_memory='True', pin_memory_device='cuda')

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(pytorch_model.parameters(), lr=1e-4)

In [137]:
def to_categorical(y, num_classes):
    return np.eye(num_classes, dtype='uint8')[y]

pytorch_model = Pytorch_BUS_Final_Model_C1().to(torch.device('cuda'))

def training_one_epoch():
    train_loss = 0
    train_correct = 0

    valid_loss = 0
    valid_correct = 0

    pytorch_model.train(True)
    for i, data in enumerate(training_loader,0):
        train_input, train_label = data
        train_input = train_input.unsqueeze(1).float().to(torch.device('cuda'))
        train_label = torch.tensor(to_categorical(train_label, 221)).float().to(torch.device('cuda'))

        if len(train_label.shape) == 1:
            train_label = train_label.unsqueeze(0)

        train_input = train_input.to(torch.device('cuda'))
        train_label = train_label.to(torch.device('cuda'))

        optimizer.zero_grad()
        train_output = pytorch_model(train_input)

        loss = loss_fn(train_output, train_label)
        loss.backward()

        optimizer.step()

        train_loss += loss.item()
        for batch_count in range(train_label.shape[0]):
            if torch.argmax(train_output[batch_count,:]) == torch.argmax(train_label[batch_count]):
                train_correct += 1

    pytorch_model.train(False)
    for i, data in enumerate(validation_loader,0):
        valid_input, valid_label = data
        valid_input = valid_input.unsqueeze(1).float().to(torch.device('cuda'))
        valid_label = torch.tensor(to_categorical(valid_label, 221)).float().to(torch.device('cuda'))

        if len(valid_label.shape) == 1:
            valid_label = valid_label.unsqueeze(0)

        valid_input = valid_input.to(torch.device('cuda'))
        valid_label = valid_label.to(torch.device('cuda'))

        valid_output = pytorch_model(valid_input)

        loss = loss_fn(valid_output, valid_label)

        valid_loss += loss.item()
        for batch_count in range(valid_label.shape[0]):
            if torch.argmax(valid_output[batch_count,:]) == torch.argmax(valid_label[batch_count]):
                valid_correct += 1
    
    print(f"Training: \n Training Accuracy: {100*train_correct/len(training_loader.dataset)}%, Average Training Loss: {train_loss/len(training_loader)}")

    print(f"Validation: \n Validation Accuracy: {100*valid_correct/len(validation_loader.dataset)}%, Average Validation Loss: {valid_loss/len(validation_loader)}")
    return train_loss, valid_loss


In [138]:
epoch_number = 0
EPOCHS = 100
optimizer = torch.optim.Adam(pytorch_model.parameters(), lr=1e-4)
for epoch in range (EPOCHS):
    print("Epoch {}:".format(epoch_number+1))
    train_loss, valid_loss = training_one_epoch()
    epoch_number += 1

Epoch 1:


C:\Users\209141\AppData\Local\Temp\ipykernel_31660\1924088813.py:90: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = self.fc1_activation(out)


Training: 
 Training Accuracy: 1.4285714285714286%, Average Training Loss: 5.394372642040253
Validation: 
 Validation Accuracy: 3.6303630363036303%, Average Validation Loss: 5.396330547332764
Epoch 2:
Training: 
 Training Accuracy: 4.8979591836734695%, Average Training Loss: 5.386608302593231
Validation: 
 Validation Accuracy: 1.3201320132013201%, Average Validation Loss: 5.395488595962524
Epoch 3:
Training: 
 Training Accuracy: 5.1020408163265305%, Average Training Loss: 5.37213659286499
Validation: 
 Validation Accuracy: 1.6501650165016502%, Average Validation Loss: 5.3900840282440186
Epoch 4:
Training: 
 Training Accuracy: 7.959183673469388%, Average Training Loss: 5.347459554672241
Validation: 
 Validation Accuracy: 3.6303630363036303%, Average Validation Loss: 5.374849939346314
Epoch 5:
Training: 
 Training Accuracy: 13.46938775510204%, Average Training Loss: 5.307069778442383
Validation: 
 Validation Accuracy: 4.9504950495049505%, Average Validation Loss: 5.364626407623291
Epoch 